In [1]:
import os
import json
import pandas as pd
import numpy as np
import warnings
import reverse_geocode
warnings.filterwarnings('ignore')

# ETL
La idea en este notebook es realizar las transformaciones correspondientes a los datos de Google Maps, tanto la metadata como los reviews. 

En este caso el alcance se limita a los 5 estados con mayor población California, Nueva York, Texas, Pensilvania y Florida. En cuanto a lo estudiado va a ser la categoria restaurantes, incluyendo todo tipo de los mismos.

In [2]:
#Generamos un diccionario con las carpetas de los archivos
ruta_review_estados={
    'New_York': 'Datasets/Google Maps/reviews-estados/review-New_York',
    'California': 'Datasets/Google Maps/reviews-estados/review-California',
    'Texas': 'Datasets/Google Maps/reviews-estados/review-Texas',
    'Florida': 'Datasets/Google Maps/reviews-estados/review-Florida',
    'Pennsylvania': 'Datasets/Google Maps/reviews-estados/review-Pennsylvania',

}

In [3]:
#Creamos un diccionario vacio para almacenar las claves de los reviews para poder filtrar los datos de metadata
gmaps_ids = {}
#Recorremos todas las rutas para cargar los datos de todos los estados
for estado, ruta_estado in ruta_review_estados.items():
#Se utiliza set para evitar duplicados
    gmaps_ids[estado] =set()
    #Se recorren todos los json de las carpetas de los reviews de los estados
    for archivo in os.listdir(ruta_estado):
        ruta_archivo=os.path.join(ruta_estado, archivo)
        print(ruta_archivo)
        with open(ruta_archivo,'r') as file:
            for linea in file:
                data = json.loads(linea)
                gmaps_ids[estado].add(data['gmap_id'])


Datasets/Google Maps/reviews-estados/review-New_York\1.json
Datasets/Google Maps/reviews-estados/review-New_York\10.json
Datasets/Google Maps/reviews-estados/review-New_York\11.json
Datasets/Google Maps/reviews-estados/review-New_York\12.json
Datasets/Google Maps/reviews-estados/review-New_York\13.json
Datasets/Google Maps/reviews-estados/review-New_York\14.json
Datasets/Google Maps/reviews-estados/review-New_York\15.json
Datasets/Google Maps/reviews-estados/review-New_York\16.json
Datasets/Google Maps/reviews-estados/review-New_York\17.json
Datasets/Google Maps/reviews-estados/review-New_York\18.json
Datasets/Google Maps/reviews-estados/review-New_York\2.json
Datasets/Google Maps/reviews-estados/review-New_York\3.json
Datasets/Google Maps/reviews-estados/review-New_York\4.json
Datasets/Google Maps/reviews-estados/review-New_York\5.json
Datasets/Google Maps/reviews-estados/review-New_York\6.json
Datasets/Google Maps/reviews-estados/review-New_York\7.json
Datasets/Google Maps/reviews-es

In [4]:
#Ruta de la carpeta metadata
ruta_metadata = 'Datasets/Google Maps/metadata-sitios'
#Diccionario para almacenar los datos de metadata
metadata = {}
#Recorremos todos los archivos de la carpeta metadata
for archivo in os.listdir(ruta_metadata):
    ruta_archivo = os.path.join(ruta_metadata, archivo)
    with open(ruta_archivo, 'r',encoding='ISO-8859-1') as file:
        for linea in file:
            data = json.loads(linea)
            #Si el gmap_id esta en el diccionario de gmaps_ids se guarda
            for estado in gmaps_ids.keys():
                if data['gmap_id'] in gmaps_ids[estado]:
                    metadata[data['gmap_id']] = data
                    #Ademas se agrega una columna con el estado al que pertenece
                    metadata[data['gmap_id']]['estado'] = estado

In [5]:
df_metadata = pd.DataFrame(metadata).T

In [6]:
df_metadata.reset_index(inplace=True)

In [7]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319881 entries, 0 to 319880
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   index             319881 non-null  object
 1   name              319879 non-null  object
 2   address           317103 non-null  object
 3   gmap_id           319881 non-null  object
 4   description       44767 non-null   object
 5   latitude          319881 non-null  object
 6   longitude         319881 non-null  object
 7   category          319632 non-null  object
 8   avg_rating        319881 non-null  object
 9   num_of_reviews    319881 non-null  object
 10  price             47766 non-null   object
 11  hours             276487 non-null  object
 12  MISC              286245 non-null  object
 13  state             279719 non-null  object
 14  relative_results  304211 non-null  object
 15  url               319881 non-null  object
 16  estado            319881 non-null  obj

In [8]:
df_metadata.isnull().sum()

index                    0
name                     2
address               2778
gmap_id                  0
description         275114
latitude                 0
longitude                0
category               249
avg_rating               0
num_of_reviews           0
price               272115
hours                43394
MISC                 33636
state                40162
relative_results     15670
url                      0
estado                   0
dtype: int64

Se eliminan todas las columnas que no aportan al proyecto ya sea por gran cantidad de nulos (price,description) o porque son innecesarias.

In [9]:
df_metadata.drop(columns=['index','description','hours','MISC','relative_results','url','price'],inplace=True)

Además se eliminan las filas con los valores faltantes en categoría ya que es de suma importancia para el proyecto.

In [10]:
df_metadata.dropna(subset='category',inplace=True)

In [11]:
df_metadata.isnull().sum()

name                  2
address            2777
gmap_id               0
latitude              0
longitude             0
category              0
avg_rating            0
num_of_reviews        0
state             40050
estado                0
dtype: int64

In [12]:
df_metadata.head(3)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,state,estado
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,[Korean restaurant],4.4,18,Open ⋅ Closes 6PM,California
1,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,[Fabric store],4.3,7,Open ⋅ Closes 5PM,California
2,Matrix International Textiles,"Matrix International Textiles, 1363 S Bonnie B...",0x80c2cf163db6bc89:0x219484e2edbcfa41,34.015505,-118.181839,[Fabric store],3.5,6,Open ⋅ Closes 5:30PM,California


Vamos a filtrar la información del dataframe por el tipo de negocio que queramos estudiar, en este caso los restaurantes. Por lo que consideramos 2 palabras clave, *restaurant* y *food* para tener las categorías mas importantes.

In [13]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319632 entries, 0 to 319880
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            319630 non-null  object
 1   address         316855 non-null  object
 2   gmap_id         319632 non-null  object
 3   latitude        319632 non-null  object
 4   longitude       319632 non-null  object
 5   category        319632 non-null  object
 6   avg_rating      319632 non-null  object
 7   num_of_reviews  319632 non-null  object
 8   state           279582 non-null  object
 9   estado          319632 non-null  object
dtypes: object(10)
memory usage: 26.8+ MB


In [63]:
df_filtered=df_metadata.copy()
#Definimos las palabras claves
palabras_claves = ['restaurant', 'food','steakhouse','mexican','pizzeria','american','asian']
#Expando categoría

#Genero una función para filtrar las palabras claves
def filter_list_by_keywords(lst, palabras_claves):
    return [word for word in lst if any(palabra.lower() in word.lower() for palabra in palabras_claves)]

df_filtered['category'] = df_filtered['category'].apply(lambda x: filter_list_by_keywords(x, palabras_claves))

In [64]:
df_filtered = df_filtered[df_filtered['category'].apply(lambda x: len(x) > 0)]

In [65]:
#abrimos el diccionario categorias
df_dicc=pd.read_excel(r'E:\Data Science\PF\Diccionario Categorías.xlsx')


In [66]:
'''#primero agrupamos por traduccion
df_dicc = df_dicc.groupby('TRADUCCION').agg({'ORIGINAL': list}).reset_index()
dicc_categorias = df_dicc.set_index('TRADUCCION')['ORIGINAL'].to_dict()'''

"#primero agrupamos por traduccion\ndf_dicc = df_dicc.groupby('TRADUCCION').agg({'ORIGINAL': list}).reset_index()\ndicc_categorias = df_dicc.set_index('TRADUCCION')['ORIGINAL'].to_dict()"

In [67]:
#hacemos un explode la columna categorias
df_filtered = df_filtered.explode('category')

In [68]:
df_filtered=df_filtered.copy()
df_filtered=df_filtered.merge(df_dicc, left_on='category', right_on='ORIGINAL', how='left')
df_filtered.drop(columns=['ORIGINAL','category'],inplace=True)
df_filtered.rename(columns={'TRADUCCION':'category'},inplace=True)
df_filtered.dropna(subset='category',inplace=True)

In [69]:
prueba['category'].value_counts()

category
Fast Food         8337
Italian           6732
East Asian        5134
Mexican           4822
American          4507
Chinese           2943
Veggie Vegan      2780
Caribbean         1504
Latin American     789
Middle East        597
Indian             472
Spanish            249
French             202
Greek              182
Jew                168
African            147
Name: count, dtype: int64

In [70]:
prueba['category'].nunique()

16

In [71]:
df_dicc['TRADUCCION'].nunique()

16

In [72]:
#Verificamos los tipo de datos de categoria
cantidad=df_filtered.explode('category')
cantidad['category'].value_counts()

category
Fast Food         8337
Italian           6732
East Asian        5134
Mexican           4822
American          4507
Chinese           2943
Veggie Vegan      2780
Caribbean         1504
Latin American     789
Middle East        597
Indian             472
Spanish            249
French             202
Greek              182
Jew                168
African            147
Name: count, dtype: int64

In [73]:
df_filtered.reset_index(drop=True, inplace=True)

Eliminamos a continuación los restaurantes que están permantemente cerrados.

In [74]:
df_filtered=df_filtered[df_filtered['state']!='Permanently closed']

Analizamos los duplicados

In [75]:
df_filtered['gmap_id'].duplicated().sum()

8958

Como vemos no tenemos dos restaurants con el mismo gmap_id por lo que esto esta en lo correcto.

In [76]:
df_filtered['name'].duplicated().sum()

15320

In [77]:
df_filtered[df_filtered['name'].duplicated()]

,name,address,gmap_id,latitude,longitude,avg_rating,num_of_reviews,state,estado,category
6,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",0x80c2d765f8c90a3d:0x16afb75943e7ad50,34.079995,-117.988951,3.7,38,Closed ⋅ Opens 6AM,California,American
7,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",0x80c2d765f8c90a3d:0x16afb75943e7ad50,34.079995,-117.988951,3.7,38,Closed ⋅ Opens 6AM,California,American
8,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",0x80c2d765f8c90a3d:0x16afb75943e7ad50,34.079995,-117.988951,3.7,38,Closed ⋅ Opens 6AM,California,Fast Food
9,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",0x80c2d765f8c90a3d:0x16afb75943e7ad50,34.079995,-117.988951,3.7,38,Closed ⋅ Opens 6AM,California,Mexican
13,Wawa,"Wawa, 870 2nd St Pike, Richboro, PA 18954",0x89c6ac87db2a14b9:0x574dceb7c72ce463,40.210823,-75.01208,3.6,34,Open 24 hours,Pennsylvania,Fast Food
...,...,...,...,...,...,...,...,...,...,...
39554,Kyuramen,"Kyuramen, 210 E 14th St, New York, NY 10003",0x89c2595ad18df43d:0x8989aa9433faaaca,40.732821,-73.986808,4.5,98,Open ⋅ Closes 11PM,New_York,East Asian
39556,Subway,"Subway, Syracuse University, 720 University Av...",0x89d9f3a1b3373329:0xeb55367bab55c39,43.042096,-76.134719,3.5,28,Open ⋅ Closes 7PM,New_York,Fast Food
39558,Luthun,"Luthun, 432 E 13th St, New York, NY 10009",0x89c2592cf2935ed9:0x6672264426649f94,40.729911,-73.981883,4.8,68,Closed ⋅ Opens 5:30PM,New_York,American
39559,Luthun,"Luthun, 432 E 13th St, New York, NY 10009",0x89c2592cf2935ed9:0x6672264426649f94,40.729911,-73.981883,4.8,68,Closed ⋅ Opens 5:30PM,New_York,Veggie Vegan


Estos duplicados corresponden a grandes cadenas de comida.

In [78]:
df_prueba=df_filtered.copy()

In [79]:
df_prueba.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30445 entries, 0 to 39564
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            30445 non-null  object
 1   address         30416 non-null  object
 2   gmap_id         30445 non-null  object
 3   latitude        30445 non-null  object
 4   longitude       30445 non-null  object
 5   avg_rating      30445 non-null  object
 6   num_of_reviews  30445 non-null  object
 7   state           28693 non-null  object
 8   estado          30445 non-null  object
 9   category        30445 non-null  object
dtypes: object(10)
memory usage: 2.6+ MB


Continuaremos con el análisis de la direccion para obtener la ciudad y el codigo postal.

In [80]:
df_filtered.head()

,name,address,gmap_id,latitude,longitude,avg_rating,num_of_reviews,state,estado,category
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,4.4,18,Open ⋅ Closes 6PM,California,East Asian
1,Studio 34 Nutrition,"Studio 34 Nutrition, 3021 A, 34th St, Lubbock,...",0x86fe6dd0642d44eb:0xeaceade94f24cc15,33.563039,-101.880718,5,14,Open ⋅ Closes 2PM,Texas,Veggie Vegan
2,Bushibushi Dimsum Place,"Bushibushi Dimsum Place, 4930 Belt Line Rd, Da...",0x864c2161e9d8649b:0x22061733a9698bcb,32.953106,-96.827689,4.9,28,Closed ⋅ Opens 11AM,Texas,Chinese
3,Dunkin',"Dunkin', 4008 Bell Blvd, Queens, NY 11361",0x89c261f60bdf13db:0x38da730e4687a97b,40.763985,-73.77143,3.5,8,Open ⋅ Closes 7PM,New_York,Fast Food
5,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",0x80c2d765f8c90a3d:0x16afb75943e7ad50,34.079995,-117.988951,3.7,38,Closed ⋅ Opens 6AM,California,Fast Food


In [81]:
#Vamos a extraer la ciudad y el código postal de la columna address
df_filtered['city'] = df_filtered['address'].str.extract(r',\s*([^,]+),\s*[A-Z]{2}\s+\d{5}', expand=False)
df_filtered['zip_code'] = df_filtered['address'].str.extract(r'(\d{5})$', expand=False)

Borramos ahora la columna *state* que no nos sirve mas y verificamos los nulos.

In [82]:
df_filtered.drop(columns='state', inplace=True)

In [83]:
df_filtered.isnull().sum()

name                 0
address             29
gmap_id              0
latitude             0
longitude            0
avg_rating           0
num_of_reviews       0
estado               0
category             0
city               130
zip_code          4442
dtype: int64

In [84]:
#Creamos una funcion para verificar la ciudad y el estado
def get_location(row):
    coordinates = [(row['latitude'], row['longitude'])]
    location = reverse_geocode.search(coordinates)[0]
    if location['country_code'] == 'US':
        return pd.Series([location['city'], location['state']])
    else:
        return pd.Series([None, None])



df_filtered[['city', 'estado']] = df_filtered.apply(get_location, axis=1)

In [85]:
df_filtered

,name,address,gmap_id,latitude,longitude,avg_rating,num_of_reviews,estado,category,city,zip_code
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,4.4,18,California,East Asian,Koreatown,90005
1,Studio 34 Nutrition,"Studio 34 Nutrition, 3021 A, 34th St, Lubbock,...",0x86fe6dd0642d44eb:0xeaceade94f24cc15,33.563039,-101.880718,5,14,Texas,Veggie Vegan,Lubbock,79410
2,Bushibushi Dimsum Place,"Bushibushi Dimsum Place, 4930 Belt Line Rd, Da...",0x864c2161e9d8649b:0x22061733a9698bcb,32.953106,-96.827689,4.9,28,Texas,Chinese,Addison,75254
3,Dunkin',"Dunkin', 4008 Bell Blvd, Queens, NY 11361",0x89c261f60bdf13db:0x38da730e4687a97b,40.763985,-73.77143,3.5,8,New York,Fast Food,Bayside,11361
5,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",0x80c2d765f8c90a3d:0x16afb75943e7ad50,34.079995,-117.988951,3.7,38,California,Fast Food,Baldwin Park,91706
...,...,...,...,...,...,...,...,...,...,...,...
39558,Luthun,"Luthun, 432 E 13th St, New York, NY 10009",0x89c2592cf2935ed9:0x6672264426649f94,40.729911,-73.981883,4.8,68,New York,American,East Village,10009
39559,Luthun,"Luthun, 432 E 13th St, New York, NY 10009",0x89c2592cf2935ed9:0x6672264426649f94,40.729911,-73.981883,4.8,68,New York,Veggie Vegan,East Village,10009
39562,Jia,"Jia, 23 Essex St, New York, NY 10002",0x89c25a29eb1373a1:0x3afffd62d2b9a1dd,40.715434,-73.990083,4.8,68,New York,East Asian,Chinatown,10002
39563,China King Express,"China King Express, 6938 Erie Rd, Derby, NY 14047",0x89d31f176b64da79:0x202faca0f650e880,42.698185,-78.988253,4.1,118,New York,Chinese,Angola,14047


Los nulos que quedan se completaran con "Sin Datos" para que el áera de análisis haga su trabajo correspondiente.

In [86]:
#Completo los nulos con "Sin Datos"
df_filtered.fillna('Sin Datos', inplace=True)

In [87]:
df_filtered

,name,address,gmap_id,latitude,longitude,avg_rating,num_of_reviews,estado,category,city,zip_code
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,4.4,18,California,East Asian,Koreatown,90005
1,Studio 34 Nutrition,"Studio 34 Nutrition, 3021 A, 34th St, Lubbock,...",0x86fe6dd0642d44eb:0xeaceade94f24cc15,33.563039,-101.880718,5.0,14,Texas,Veggie Vegan,Lubbock,79410
2,Bushibushi Dimsum Place,"Bushibushi Dimsum Place, 4930 Belt Line Rd, Da...",0x864c2161e9d8649b:0x22061733a9698bcb,32.953106,-96.827689,4.9,28,Texas,Chinese,Addison,75254
3,Dunkin',"Dunkin', 4008 Bell Blvd, Queens, NY 11361",0x89c261f60bdf13db:0x38da730e4687a97b,40.763985,-73.771430,3.5,8,New York,Fast Food,Bayside,11361
5,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",0x80c2d765f8c90a3d:0x16afb75943e7ad50,34.079995,-117.988951,3.7,38,California,Fast Food,Baldwin Park,91706
...,...,...,...,...,...,...,...,...,...,...,...
39558,Luthun,"Luthun, 432 E 13th St, New York, NY 10009",0x89c2592cf2935ed9:0x6672264426649f94,40.729911,-73.981883,4.8,68,New York,American,East Village,10009
39559,Luthun,"Luthun, 432 E 13th St, New York, NY 10009",0x89c2592cf2935ed9:0x6672264426649f94,40.729911,-73.981883,4.8,68,New York,Veggie Vegan,East Village,10009
39562,Jia,"Jia, 23 Essex St, New York, NY 10002",0x89c25a29eb1373a1:0x3afffd62d2b9a1dd,40.715434,-73.990083,4.8,68,New York,East Asian,Chinatown,10002
39563,China King Express,"China King Express, 6938 Erie Rd, Derby, NY 14047",0x89d31f176b64da79:0x202faca0f650e880,42.698185,-78.988253,4.1,118,New York,Chinese,Angola,14047


In [88]:
#renombo las columnas correspondientes
df_filtered.rename(columns={'estado':'state'},inplace=True)

In [89]:
#reordeno las columnas
df_filtered = df_filtered[['gmap_id', 'name', 'address', 'city', 'state', 'zip_code', 'latitude', 'longitude', 'category']]

In [90]:
df_filtered.head()

,gmap_id,name,address,city,state,zip_code,latitude,longitude,category
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",Koreatown,California,90005,34.058092,-118.292130,East Asian
1,0x86fe6dd0642d44eb:0xeaceade94f24cc15,Studio 34 Nutrition,"Studio 34 Nutrition, 3021 A, 34th St, Lubbock,...",Lubbock,Texas,79410,33.563039,-101.880718,Veggie Vegan
2,0x864c2161e9d8649b:0x22061733a9698bcb,Bushibushi Dimsum Place,"Bushibushi Dimsum Place, 4930 Belt Line Rd, Da...",Addison,Texas,75254,32.953106,-96.827689,Chinese
3,0x89c261f60bdf13db:0x38da730e4687a97b,Dunkin',"Dunkin', 4008 Bell Blvd, Queens, NY 11361",Bayside,New York,11361,40.763985,-73.771430,Fast Food
5,0x80c2d765f8c90a3d:0x16afb75943e7ad50,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",Baldwin Park,California,91706,34.079995,-117.988951,Fast Food


Se verifican finalmente que no haya duplicados

In [91]:
#Cuento los duplicados en gmap_id
df_filtered['gmap_id'].duplicated().sum()

8958

In [92]:
#Modifíco la columna state para que contenga la abreviacíon de los estados
def abreviacion_estado(row):
    if row['state'] == 'New York':
        return 'NY'
    elif row['state'] == 'California':
        return 'CA'
    elif row['state'] == 'Texas':
        return 'TX'
    elif row['state'] == 'Florida':
        return 'FL'
    elif row['state'] == 'Pennsylvania':
        return 'PA'
    else:
        return 'Sin Datos'

df_filtered['state'] = df_filtered.apply(abreviacion_estado, axis=1)

In [93]:
df_filtered['state'].value_counts()

state
NY           10036
CA            6790
PA            5042
TX            4247
FL            4246
Sin Datos       84
Name: count, dtype: int64

In [94]:
df_restaurantes_clean = df_filtered.copy()

In [95]:
df_restaurantes_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30445 entries, 0 to 39564
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   gmap_id    30445 non-null  object 
 1   name       30445 non-null  object 
 2   address    30445 non-null  object 
 3   city       30445 non-null  object 
 4   state      30445 non-null  object 
 5   zip_code   30445 non-null  object 
 6   latitude   30445 non-null  float64
 7   longitude  30445 non-null  float64
 8   category   30445 non-null  object 
dtypes: float64(2), object(7)
memory usage: 2.3+ MB


In [96]:
df_restaurantes_clean.head(3)

,gmap_id,name,address,city,state,zip_code,latitude,longitude,category
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",Koreatown,CA,90005,34.058092,-118.292130,East Asian
1,0x86fe6dd0642d44eb:0xeaceade94f24cc15,Studio 34 Nutrition,"Studio 34 Nutrition, 3021 A, 34th St, Lubbock,...",Lubbock,TX,79410,33.563039,-101.880718,Veggie Vegan
2,0x864c2161e9d8649b:0x22061733a9698bcb,Bushibushi Dimsum Place,"Bushibushi Dimsum Place, 4930 Belt Line Rd, Da...",Addison,TX,75254,32.953106,-96.827689,Chinese


In [97]:
df_restaurantes_clean.to_parquet('Datos limpios/restaurantes_clean.parquet')

In [99]:
df_restaurantes_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30445 entries, 0 to 39564
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   gmap_id    30445 non-null  object 
 1   name       30445 non-null  object 
 2   address    30445 non-null  object 
 3   city       30445 non-null  object 
 4   state      30445 non-null  object 
 5   zip_code   30445 non-null  object 
 6   latitude   30445 non-null  float64
 7   longitude  30445 non-null  float64
 8   category   30445 non-null  object 
dtypes: float64(2), object(7)
memory usage: 2.3+ MB


In [50]:
df_dummies_restaurantes=df_restaurantes_clean[['gmap_id','category']]
categoria=df_dummies_restaurantes['category'].str.get_dummies(sep=',')

In [51]:
df_dummies_restaurantes=pd.concat([df_dummies_restaurantes,categoria],axis=1)

In [52]:
df_dummies_restaurantes.reset_index(drop=True, inplace=True)

In [53]:
df_dummies_restaurantes.to_parquet('Datos limpios/dummies_restaurantes2.parquet')

In [54]:
df_restaurantes_clean['state'].value_counts()

state
NY           10120
CA            6907
PA            5337
TX            4744
FL            4234
Sin Datos       98
Name: count, dtype: int64

In [2]:
df=pd.read_parquet(r'Datos limpios\restaurantes_clean.parquet')

In [4]:
df['city'].value_counts()

city
Times Square     302
East Village     222
Chinatown        219
Manhattan        178
Gramercy Park    170
                ... 
Bostonia           1
Snyder             1
Roby               1
Cherry Valley      1
Sheffield          1
Name: count, Length: 3367, dtype: int64